# **KawsAI - Un Modelo de IA para Matching de ChambeaYa**

En este notebook se guarda el proceso que se hizo para crear el modelo de IA KawsAI. El objetivo de KawsAI, es hacer el matching con la mejor mype a cada estudiante. Así, el estudiante y la mype pueden conectar para poder trabajar juntos.

### **Versión:**
1.0

### **Objetivo:**
El objetivo de KawsAI es lograr hacer el mejor match entre estudiantes y mypes. Este match se va a basar en datos que ChambeaYa va a recopilar de ambos lados a traves de su página web. El objetivo principal es lograr indicar a un estudiante cual es el puesto que ofrece una mype que mas coincide con sus conocimientos, y así aumentar mucho las probabilidades de conexión.

### **Descripción de los Datasets**
Para la versión 1.0 vamos a usar datasets simulados con la ayuda de la IA de ChatGPT. Se le pidió a ChatGPT simular 2 datasets en forma de tablas y en formato .csv, que son estudiantes_test.csv y mypes_test.csv. Cada una de las columnas de estos datasets contiene información que se va a solicitar a los usuarios ingresar en nuestra plataforma.

##### Descrición de columnas de estudiantes:

##### Descripción de columnas de mypes:

### **Índice**



In [38]:
# Primeramente importamos las librerias
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np
import spacy
from spacy.lang.es.stop_words import STOP_WORDS as spacy_stopwords
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 11.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [40]:
# Ahora leemos los csv y los guardamos en dataframes de pandas
estudiantes = pd.read_csv("../datasets/estudiantes_test.csv")
mypes = pd.read_csv("../datasets/mypes_test.csv")

In [41]:
nlp = spacy.load("es_core_news_sm")

def preprocesar(text):
  text = text.lower()
  text = re.sub(r"[^\w\s]", " ", text)     # Quita puntuación
  text = re.sub(r"\s+", " ", text).strip() # Espacios múltiples

  doc = nlp(text)
  tokens = []

  for token in doc:
    if token.is_stop or token.is_punct or token.is_space:
      continue
    if token.like_num:
      continue
    tokens.append(token.lemma_)
  return " ".join(tokens)

# 1er Filtro - Filtrado por columnas específicas

In [42]:
def filtrar_por_columnas(indice_estudiante, estudiantes, mypes):
    estudiante = estudiantes.iloc[indice_estudiante]
    modalidad = estudiante["modalidad_preferida"]
    horas_disponibles = estudiante["disponibilidad_semanal"]

    print(f"{modalidad} {horas_disponibles}")
    # Filtrar por modalidad y horas
    mypes_aux = mypes[
        (mypes["modalidad_proyecto"] == modalidad) &
        (mypes["horas_requeridas"] <= horas_disponibles)
    ]
    return mypes_aux.reset_index(drop=True)


In [7]:
mypes_filtradas = filtrar_por_columnas(6, estudiantes, mypes)

Presencial 25


In [19]:
mypes_filtradas.shape

(665, 8)

# Ahora preprocesamos solo el estudiante y las mypes_filtradas

In [9]:
# Para el estudiante
estudiante = estudiantes.iloc[6]
estudiante["skill_estrella"] = preprocesar(estudiante["skill_estrella"])
estudiante["motivación_principal"] = preprocesar(estudiante["motivación_principal"])
estudiante["areas_interes"] = preprocesar(estudiante["areas_interes"])

/tmp/ipykernel_189448/3188254507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["skill_estrella"] = preprocesar(estudiante["skill_estrella"])
/tmp/ipykernel_189448/3188254507.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["motivación_principal"] = preprocesar(estudiante["motivación_principal"])
/tmp/ipykernel_189448/3188254507.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["areas_interes"

In [10]:
estudiante

nombre                                                         Carlos López
correo                                         carlos.lópez21@correo.edu.pe
ubicación                               Trujillo, La Libertad, Víctor Larco
skill_estrella               diseño material publicitario adobe illustrator
motivación_principal                                               aprender
hobbies                                              Voluntariado ambiental
disponibilidad_semanal                                                   25
modalidad_preferida                                              Presencial
areas_interes             fundamento diseño gráfico metodología básico d...
carrera                                                      Diseño Gráfico
ciclo                                                                     3
Name: 6, dtype: object

In [11]:
# Para la mype
mypes_filtradas["área_reto"] = mypes_filtradas["área_reto"].apply(preprocesar)
mypes_filtradas["descripción_reto"] = mypes_filtradas["descripción_reto"].apply(preprocesar)
mypes_filtradas["habilidades_requeridas"] = mypes_filtradas["habilidades_requeridas"].apply(preprocesar)
mypes_filtradas["cultura_empresa"] = mypes_filtradas["cultura_empresa"].apply(preprocesar)

In [12]:
mypes_filtradas.head()

,nombre_empresa,RUC,área_reto,descripción_reto,habilidades_requeridas,cultura_empresa,horas_requeridas,modalidad_proyecto
0,Soluciones Marketing Corporación,70925802916,legal,necesitar elaborar política interno claro concisa,redacción legal derecho empresarial análisis c...,empresa joven tecnológico,20,Presencial
1,Progreso Marketing SRL,34980195388,marketing,buscar desarrollar estrategia contenido captar...,seo googlar ads marketing digital,organización orientado impacto social,20,Presencial
2,Progreso Exportaciones SAC,87188857461,legal,necesitar elaborar política interno claro concisa,excel normativo peruano derecho empresarial an...,startup dinámico colaborativo,25,Presencial
3,Servicios Exportaciones Corporación,38319868934,recurso humano,mejorar proceso reclutamiento selección,psicología organizacional comunicación efectiv...,empresa familiar enfoque innovación social,25,Presencial
4,Comercial Diseño SPA,96105252980,diseño,rediseñar página web mejorar experiencia usuario,figma html css adobe illustrator,organización orientado impacto social,25,Presencial


# 2do Filtro - Creamos los vectores TF-IDF para el subset de mypes

In [ ]:
def concatenar_texto_estudiante(estudiantes):
  return (
      str(estudiantes["skill_estrella"]) + " " +
      str(estudiantes["motivación_principal"]) + " " +
      str(estudiantes["areas_interes"])
  )

def concatenar_texto_mypes(mypes):
  return (
      str(mypes["área_reto"]) + " " +
      str(mypes["descripción_reto"]) + " " +
      str(mypes["habilidades_requeridas"]) + " " +
      str(mypes["cultura_empresa"])
  )

def calcular_similitud_texto(estudiante, mypes_filtradas):
    # primero concatenamos todo
    texto_estudiante = concatenar_texto_estudiante(estudiante)
    textos_mypes = mypes_filtradas.apply(concatenar_texto_mypes, axis=1)

    # Ahora juntamos el texto estudiante con el texto de las mipes con lista
    corpus = [texto_estudiante] + textos_mypes.tolist()

    # Hacemos la vectorización TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Ahora hacemos el cosine similarity
    similitudes = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

    # Ahora usamos una columna de similitud
    mypes_filtradas = mypes_filtradas.copy()
    mypes_filtradas["similitud"] = similitudes

    # Ordenamos las mypes_filtradas de mayor a menor
    mypes_filtradas = mypes_filtradas.sort_values(by="similitud", ascending=False).reset_index(drop=True)

    return mypes_filtradas

In [15]:
# 2. Filtrar MYPEs compatibles
mypes_filtradas = filtrar_por_columnas(6, estudiantes, mypes)

# 3. Calcular similitudes
mypes_con_similitud = calcular_similitud_texto(estudiante, mypes_filtradas)

# 4. Ver resultado
mypes_con_similitud[["RUC", "similitud"]]


Presencial 25


,RUC,similitud
0,53206862288,0.391569
1,14894252875,0.388043
2,33820574064,0.381881
3,81227971577,0.381151
4,45831847091,0.370377
...,...,...
660,37965841620,0.000000
661,14366201374,0.000000
662,30804139665,0.000000
663,92231914025,0.000000


In [16]:
estudiantes.iloc[6]

nombre                                                         Carlos López
correo                                         carlos.lópez21@correo.edu.pe
ubicación                               Trujillo, La Libertad, Víctor Larco
skill_estrella            Diseño de material publicitario en Adobe Illus...
motivación_principal                                               Aprender
hobbies                                              Voluntariado ambiental
disponibilidad_semanal                                                   25
modalidad_preferida                                              Presencial
areas_interes             Fundamentos de Diseño Gráfico, Metodologías bá...
carrera                                                      Diseño Gráfico
ciclo                                                                     3
Name: 6, dtype: object

In [17]:
mypes_filtradas[mypes_filtradas['RUC']==53206862288]

,nombre_empresa,RUC,área_reto,descripción_reto,habilidades_requeridas,cultura_empresa,horas_requeridas,modalidad_proyecto
239,Estrategias Legal SRL,53206862288,Diseño,Requerimos material gráfico para redes y prese...,"Adobe Illustrator, HTML/CSS",Startup dinámica y colaborativa,20,Presencial
